# Accessibility Mapping in R

Related papers:

- https://www.nature.com/articles/nature25181#Sec2
- https://www.nature.com/articles/s41597-019-0265-5#Tab1
- https://www.nature.com/articles/s41591-020-1059-1#Sec2

Based on code from:

- https://malariaatlas.org/wp-content/uploads/accessibility-2020/R_generic_accessibilty_mapping_script_2020.r
- https://gist.github.com/giacfalk/fe2033d1c3a5c841d669d7ccdfe9e3d7

Global friction surface available here:
- https://malariaatlas.org/project-resources/accessibility-to-healthcare/

**How to run**

1. Prepare friction map for area of interest (motorized or foot)
2. Prepare point locations of interest (.csv must have the uid, X_COORDS, Y_COORDS as columns)
2. Open anaconda
2. Activate R_tools environment
3. Start jupyter notebook

In [17]:
## For the first part (accessibility map)
library(sf)
library(raster)
library(gdistance)

Linking to GEOS 3.9.0, GDAL 3.2.1, PROJ 7.2.1
Loading required package: sp
Loading required package: igraph

Attaching package: 'igraph'

The following object is masked from 'package:raster':

    union

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union

Loading required package: Matrix

Attaching package: 'gdistance'

The following object is masked from 'package:igraph':

    normalize



## Accessibility map generation 

In [18]:
# User Defined Variables - used if clipping from the global layer, if no clipping is needed, see lines 66-67 (currently commented out).
# This could also be accomplished by importing a shapefile (for example) 
# Geographic Coordinates (WGS84)
#left   <- -2.0
#right  <- 0.0
#bottom <- 50.0
#top    <- 52.0
transition.matrix.exists.flag <- 0 # if the geo-corrected graph has already been made, this can save time.  Uses the same T.GC.filename as specified using the T.GC.filename variable.

In [19]:
# Input Files
# Note: the alternate, 'walking_only' friction surface is named friction_surface_2019_v51_walking_only.tif
friction.surface.filename <- 'GIS_data/motorized_friction_surface_Moz.tif'
point.filename1 <- 'GIS_data/Key_locations_for_Distance/airports/airports.csv' # Just 2 columns.  Structured as [X_COORD, Y_COORD] aka [LONG, LAT].  Use a header.
point.filename2 <- 'GIS_data/Key_locations_for_Distance/ports/ports.csv' # Just 2 columns.  Structured as [X_COORD, Y_COORD] aka [LONG, LAT].  Use a header.
point.filename3 <- 'GIS_data/Key_locations_for_Distance/railways/railways.csv' # Just 2 columns.  Structured as [X_COORD, Y_COORD] aka [LONG, LAT].  Use a header.
point.filename4 <- 'GIS_data/Key_locations_for_Distance/Cities_Towns/cities.csv' # Just 2 columns.  Structured as [X_COORD, Y_COORD] aka [LONG, LAT].  Use a header.
point.filename5 <- 'GIS_data/Key_locations_for_Distance/Cities_Towns/capital.csv' # Just 2 columns.  Structured as [X_COORD, Y_COORD] aka [LONG, LAT].  Use a header.

In [20]:
# Output Files
T.filename <- 'GIS_data/Moz_suitability/study.area.T.rds'
T.GC.filename <- 'GIS_data/Moz_suitability/study.area.T.GC.rds'
output.filename1 <- 'GIS_data/Moz_suitability/study.area.accessibility_airports.tif'
output.filename2 <- 'GIS_data/Moz_suitability/study.area.accessibility_ports.tif'
output.filename3 <- 'GIS_data/Moz_suitability/study.area.accessibility_railways.tif'
output.filename4 <- 'GIS_data/Moz_suitability/study.area.accessibility_cities.tif'
output.filename5 <- 'GIS_data/Moz_suitability/study.area.accessibility_capital.tif'

In [21]:
# Read in the points table
points1 <- read.csv(file = point.filename1)
points2 <- read.csv(file = point.filename2)
points3 <- read.csv(file = point.filename3)
points4 <- read.csv(file = point.filename4)
points5 <- read.csv(file = point.filename5)

# Fetch the number of points
temp1 <- dim(points1)
n.points1 <- temp1[1]
# Fetch the number of points
temp2 <- dim(points2)
n.points2 <- temp2[1]
# Fetch the number of points
temp3 <- dim(points3)
n.points3 <- temp3[1]
# Fetch the number of points
temp4 <- dim(points4)
n.points4 <- temp4[1]
# Fetch the number of points
temp5 <- dim(points5)
n.points5 <- temp5[1]

head(points1, 4) ## show 4 first centres

X_COORD,Y_COORD
35.42163,-24.22472
34.56107,-21.16255
33.50113,-25.19959
33.23860,-25.26599


In [22]:
#  Define the spatial template
#friction <- raster(friction.surface.filename)
#fs1 <- crop(friction, extent(left, right, bottom, top))
# Use the following line instead of the preceding 2 if clipping is not needed (i.e., to run globally), but be warned that trying this will far exceed the computational capacity available to most users.
fs1 <- raster(friction.surface.filename) 

In [23]:
# Make the graph and the geocorrected version of the graph (or read in the latter).
if (transition.matrix.exists.flag == 1) {
    # Read in the transition matrix object if it has been pre-computed
    T.GC <- readRDS(T.GC.filename)
} else {
    # Make and geocorrect the transition matrix (i.e., the graph)
    T <- transition(fs1, function(x) 1/mean(x), 8) # RAM intensive, can be very slow for large areas
                    saveRDS(T, T.filename)
                    T.GC <- geoCorrection(T)                    
                    saveRDS(T.GC, T.GC.filename)
                    }

In [24]:
# Convert the points into a matrix
xy.data.frame1 <- data.frame()
xy.data.frame1[1:n.points1,1] <- points1[,1]
xy.data.frame1[1:n.points1,2] <- points1[,2]
xy.matrix1 <- as.matrix(xy.data.frame1)

# Convert the points into a matrix
xy.data.frame2 <- data.frame()
xy.data.frame2[1:n.points2,1] <- points2[,1]
xy.data.frame2[1:n.points2,2] <- points2[,2]
xy.matrix2 <- as.matrix(xy.data.frame2)

# Convert the points into a matrix
xy.data.frame3 <- data.frame()
xy.data.frame3[1:n.points3,1] <- points3[,1]
xy.data.frame3[1:n.points3,2] <- points3[,2]
xy.matrix3 <- as.matrix(xy.data.frame3)

# Convert the points into a matrix
xy.data.frame4 <- data.frame()
xy.data.frame4[1:n.points4,1] <- points4[,1]
xy.data.frame4[1:n.points4,2] <- points4[,2]
xy.matrix4 <- as.matrix(xy.data.frame4)

# Convert the points into a matrix
xy.data.frame5 <- data.frame()
xy.data.frame5[1:n.points5,1] <- points5[,1]
xy.data.frame5[1:n.points5,2] <- points5[,2]
xy.matrix5 <- as.matrix(xy.data.frame5)

In [25]:
# Run the accumulated cost algorithm to make the final output map. This can be quite slow (potentially hours).
temp.raster1 <- accCost(T.GC, xy.matrix1)
temp.raster2 <- accCost(T.GC, xy.matrix2)
temp.raster3 <- accCost(T.GC, xy.matrix3)
temp.raster4 <- accCost(T.GC, xy.matrix4)
temp.raster5 <- accCost(T.GC, xy.matrix5)

In [26]:
# Write the resulting raster
writeRaster(temp.raster1, output.filename1)
writeRaster(temp.raster2, output.filename2)
writeRaster(temp.raster3, output.filename3)
writeRaster(temp.raster4, output.filename4)
writeRaster(temp.raster5, output.filename5)

## Catchment area & population summary extraction

In [1]:
## For the second part (catchment), which does not work at the moment
#library(tidyverse)
#library(data.table)
#library(exactextractr)
#library(s2)
#library(stars)

In [2]:
#minutes_cluster <- 15 # minutes of travel time around the facility, customizable
#pop <- raster("GIS_data/hrsl_moz_pop.tif") # gridded population raster of reference

In [3]:
##points <- read_sf("GIS_data/data_points.shp") # sf shapefile of healthcare facilities (points)

In [4]:
##points$id <- 1:nrow(points)

In [5]:
#coordinates(points)=~X_COORD+Y_COORD
#points_df <- st_as_sf(points)

In [6]:
#points_df

In [7]:
##class(points$fid)
##class(points$Population)
##class(points$ElecPop)
##class(points$NightLight)
##class(points$Area)
##class(points$id)
##class(points$Country)
##class(points_df$X_COORD)
##class(points_df$Y_COORD)
#class(points_df$uid_Q001) 
#class(points_df$geometry) 

In [8]:
##class(points_df$X_COORD) <- 'numeric'
##class(points_df$Y_COORD) <- 'numeric'
##class(points_df$id) 
##class(points_df$geometry) <- 'sfc'

In [9]:
##points$fid <- as.integer(points$fid)

In [10]:
##names(points)

In [11]:
###WGScoor <- points
###coordinates(WGScoor)=~X_COORD+Y_COORD
###proj4string(WGScoor)<- CRS("+proj=longlat +datum=WGS84")
###points_shp <- spTransform(WGScoor,CRS("+proj=longlat"))

In [12]:
## create catchment areas
#
#functpop <-lapply(1:nrow(points_df),function(i){
#    id_exp = points_df[i, ]$uid_Q001
#    xy.matrix <-st_coordinates(points_df[i, ])
#    servedpop <- accCost(T.GC, xy.matrix)
#    servedpop[servedpop>minutes_cluster] <- NA
#    servedpop <- trim(servedpop)
#    servedpop <- stars::st_as_stars(servedpop)
#    servedpop <- st_as_sf(servedpop)
#    p = servedpop %>% summarise()
#    p = st_sf(p)
#    p$uid_Q001 = id_exp
#    p <- st_cast(p, "MULTIPOLYGON")
#    p
#  })

In [13]:
#pol <- sf::st_as_sf(data.table::rbindlist(functpop))

In [14]:
## extract population and redistributed among overlapping catchment areas
#
#pp = lapply(1:nrow(pol),function(i){
#  b<-st_intersects(pol[i,], pol[-i,])
#  a<-sf::st_touches(pol[i,], pol[-i,])
#  ifelse(length(setdiff(b[1][[1]], a[1][[1]]))==0, exact_extract(pop, pol[i,], 'sum'), exact_extract(pop, pol[i,], 'sum')*ifelse(length(as.numeric(st_area(st_difference(pol[i,], st_union(pol[setdiff(b[1][[1]], a[1][[1]]),])))))!=0, as.numeric(st_area(st_difference(pol[i,], st_union(pol[setdiff(b[1][[1]], a[1][[1]]),]))))/as.numeric(st_area(pol[i,])), 0) + exact_extract(pop, pol[i,], 'sum')*ifelse(length(as.numeric(st_area(st_intersection(pol[i,], st_union(pol[setdiff(b[1][[1]], a[1][[1]]),]))))!=0), as.numeric(st_area(st_intersection(pol[i,], st_union(pol[setdiff(b[1][[1]], a[1][[1]]),]))))/as.numeric(st_area(pol[i,]))/length(setdiff(b[1][[1]], a[1][[1]])),0))
#})
#
#
#pp = do.call(rbind, pp)

In [15]:
## final output: catchment area with population
#pol$population = pp
#
#write_sf(pol, "HF_catchment_areas_with_pop_Moz_15min.gpkg")

In [16]:
##library(ncdf4)
##fn <- "C:\\Users\\alexl\\Dropbox\\KTH-dESA\\Projects\\World Bank - Vivid Agricultural Demand SSA\\Water table\\AFRICA_WTD_annualmean.nc"
##nc <- nc_open(fn)
##print (nc)